# Telecom X – Parte 2: Predicción de Cancelación (Churn)  
Incluye: carga desde API JSON, limpieza, análisis exploratorio (correlación), preprocesado, entrenamiento de **Regresión Logística** y **Random Forest**, evaluación (accuracy, precision, recall, F1, AUC, matriz de confusión), interpretación (importancia y coeficientes) y conclusiones.


In [ ]:
# --- Inicio de celda: explicación de la lógica de esta sección ---
# (Opcional) Instalar imbalanced-learn para SMOTE si quieres usarlo.
# Ejecuta esta celda solo si necesitas SMOTE.
# !pip -q install imbalanced-learn
print("Si necesitas SMOTE para balancear clases, descomenta la línea de pip install.")
# --- Fin de celda ---

In [ ]:
# --- Inicio de celda: explicación de la lógica de esta sección ---
# Importación de librerías necesarias
import warnings
warnings.filterwarnings("ignore")

# Importación de librerías necesarias
import pandas as pd
# Importación de librerías necesarias
import numpy as np
# Importación de librerías necesarias
import matplotlib.pyplot as plt
# Importación de librerías necesarias
import seaborn as sns

# Importación de librerías necesarias
from sklearn.model_selection import train_test_split
# Importación de librerías necesarias
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# Importación de librerías necesarias
from sklearn.compose import ColumnTransformer, make_column_selector as selector
# Importación de librerías necesarias
from sklearn.pipeline import Pipeline
# Importación de librerías necesarias
from sklearn.linear_model import LogisticRegression
# Importación de librerías necesarias
from sklearn.ensemble import RandomForestClassifier
# Importación de librerías necesarias
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, classification_report, roc_auc_score, roc_curve)

# Asignación de variable o resultado intermedio
RANDOM_STATE = 42

# Definición de una función
def make_ohe():
    """Create OneHotEncoder with dense output in a way compatible with multiple sklearn versions."""
    try:
# Asignación de variable o resultado intermedio
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
# Asignación de variable o resultado intermedio
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

# --- Fin de celda ---

In [ ]:
# --- Inicio de celda: explicación de la lógica de esta sección ---
# 1) Cargar datos desde API JSON y aplanar campos tipo dict
# Asignación de variable o resultado intermedio
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"
# Asignación de variable o resultado intermedio
df = pd.read_json(url)

# Definición de una función
def flatten_dataframe(df_):
# Asignación de variable o resultado intermedio
    df_flat = df_.copy()
# Asignación de variable o resultado intermedio
    cols_to_expand = [c for c in df_flat.columns if df_flat[c].apply(lambda x: isinstance(x, dict)).any()]
# Bucle para iterar sobre datos
    for c in cols_to_expand:
# Asignación de variable o resultado intermedio
        expanded = pd.json_normalize(df_flat[c])
# Asignación de variable o resultado intermedio
        expanded.columns = [f"{c}_{subc}" for subc in expanded.columns]
# Asignación de variable o resultado intermedio
        df_flat = df_flat.drop(columns=[c]).join(expanded)
    return df_flat

# Asignación de variable o resultado intermedio
df = flatten_dataframe(df)
print('Shape raw:', df.shape)
df.head(3)
# --- Fin de celda ---

In [ ]:
# --- Inicio de celda: explicación de la lógica de esta sección ---
# 2) Limpieza inicial: eliminar duplicados, mapear binarios y tipos
# Eliminar duplicados considerando representación en string (maneja dicts)
# Asignación de variable o resultado intermedio
df = df.loc[~df.astype(str).duplicated()].reset_index(drop=True)

# Mapeo robusto de valores binarios que podrían aparecer
# Asignación de variable o resultado intermedio
map_yes = {"yes", "sí", "si", "true", "1", "y"}
# Asignación de variable o resultado intermedio
map_no  = {"no", "false", "0", "n"}

# Definición de una función
def binarize(v):
# Condición para ejecutar código según un criterio
    if isinstance(v, str):
# Asignación de variable o resultado intermedio
        t = v.strip().lower()
# Condición para ejecutar código según un criterio
        if t in map_yes: return 1
# Condición para ejecutar código según un criterio
        if t in map_no:  return 0
    return v

# Asignación de variable o resultado intermedio
df = df.applymap(binarize)

# Intentar convertir object a numérico donde aplique, conservar strings si no se puede
# Bucle para iterar sobre datos
for col in df.columns:
# Condición para ejecutar código según un criterio
    if df[col].dtype == 'object':
# Asignación de variable o resultado intermedio
        df[col] = pd.to_numeric(df[col], errors='ignore')

# Imputación diferenciada: numéricos -> 0 ; categóricas -> 'missing'
# Asignación de variable o resultado intermedio
num_cols_tmp = df.select_dtypes(include=[np.number]).columns.tolist()
# Asignación de variable o resultado intermedio
cat_cols_tmp = [c for c in df.columns if c not in num_cols_tmp]

# Condición para ejecutar código según un criterio
if num_cols_tmp:
# Asignación de variable o resultado intermedio
    df[num_cols_tmp] = df[num_cols_tmp].fillna(0)
# Condición para ejecutar código según un criterio
if cat_cols_tmp:
# Asignación de variable o resultado intermedio
    df[cat_cols_tmp] = df[cat_cols_tmp].fillna('missing').astype(str)

# Feature opcional
# Condición para ejecutar código según un criterio
if 'MonthlyCharges' in df.columns:
# Asignación de variable o resultado intermedio
    df['Cuentas_Diarias'] = df['MonthlyCharges'] / 30

print('Shape after cleaning:', df.shape)
df.head(3)
# --- Fin de celda ---

In [ ]:
# --- Inicio de celda: explicación de la lógica de esta sección ---
# 3) EDA rápido: revisar objetivo y correlaciones numéricas
# Asignación de variable o resultado intermedio
target_col = 'Churn'
# Condición para ejecutar código según un criterio
if target_col not in df.columns:
    raise ValueError(f"No encontré la columna objetivo '{target_col}' en el dataframe.")

# Asegurar que target sea 0/1
# Asignación de variable o resultado intermedio
df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
# Condición para ejecutar código según un criterio
if df[target_col].isna().any():
    raise ValueError("La columna target no pudo convertirse completamente a numérico. Revisa valores atípicos.")

print('Distribución de clases (counts):')
print(df[target_col].value_counts())
print('\nProporción (normalize):')
# Asignación de variable o resultado intermedio
print(df[target_col].value_counts(normalize=True))

# Correlación numérica con Churn
# Asignación de variable o resultado intermedio
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Condición para ejecutar código según un criterio
if target_col in num_cols:
    num_cols.remove(target_col)
# Asignación de variable o resultado intermedio
corr_with_y = df[num_cols + [target_col]].corr()[target_col].drop(target_col).sort_values(key=lambda x: x.abs(), ascending=False)
print('\nTop correlaciones (numéricas) con Churn:')
display(corr_with_y.head(15))
# --- Fin de celda ---

In [ ]:
# --- Inicio de celda: explicación de la lógica de esta sección ---
# 4) Train/test split
# Asignación de variable o resultado intermedio
X = df.drop(columns=[target_col])
# Asignación de variable o resultado intermedio
y = df[target_col].astype(int)

# Asignación de variable o resultado intermedio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=RANDOM_STATE, stratify=y)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

# Detectar columnas categóricas (object / string), y numéricas
# Asignación de variable o resultado intermedio
cat_cols = selector(dtype_include=object)(X_train)
# Asignación de variable o resultado intermedio
num_cols = selector(dtype_include=np.number)(X_train)

# Forzar categóricas a string (evitar mezcla de tipos)
# Condición para ejecutar código según un criterio
if len(cat_cols)>0:
# Asignación de variable o resultado intermedio
    X_train[cat_cols] = X_train[cat_cols].fillna('missing').astype(str)
# Asignación de variable o resultado intermedio
    X_test[cat_cols]  = X_test[cat_cols].fillna('missing').astype(str)

print('\nNuméricas:', len(num_cols), ' Categóricas:', len(cat_cols))
# --- Fin de celda ---

In [ ]:
# --- Inicio de celda: explicación de la lógica de esta sección ---
# 5) Preprocesamiento y pipelines
# Asignación de variable o resultado intermedio
ohe = make_ohe()

# Asignación de variable o resultado intermedio
preprocess_scaled = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', ohe, cat_cols)
# Asignación de variable o resultado intermedio
], remainder='drop')

# Asignación de variable o resultado intermedio
preprocess_unscaled = ColumnTransformer(transformers=[
    ('num', 'passthrough', num_cols),
    ('cat', ohe, cat_cols)
# Asignación de variable o resultado intermedio
], remainder='drop')

# Asignación de variable o resultado intermedio
models = {
# Asignación de variable o resultado intermedio
    'LogisticRegression': Pipeline(steps=[
        ('prep', preprocess_scaled),
# Asignación de variable o resultado intermedio
        ('clf', LogisticRegression(max_iter=2000, random_state=RANDOM_STATE))
    ]),
# Asignación de variable o resultado intermedio
    'RandomForest': Pipeline(steps=[
        ('prep', preprocess_unscaled),
# Asignación de variable o resultado intermedio
        ('clf', RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE))
    ])
}

models.keys()
# --- Fin de celda ---

In [ ]:
# --- Inicio de celda: explicación de la lógica de esta sección ---
# 6) Entrenar y evaluar
# Definición de una función
def eval_and_plot(name, model, X_te, y_te):
# Asignación de variable o resultado intermedio
    preds = model.predict(X_te)
# Asignación de variable o resultado intermedio
    probs = None
    try:
# Asignación de variable o resultado intermedio
        probs = model.predict_proba(X_te)[:,1]
    except Exception:
        pass

# Asignación de variable o resultado intermedio
    acc  = accuracy_score(y_te, preds)
# Asignación de variable o resultado intermedio
    prec = precision_score(y_te, preds, zero_division=0)
# Asignación de variable o resultado intermedio
    rec  = recall_score(y_te, preds, zero_division=0)
# Asignación de variable o resultado intermedio
    f1   = f1_score(y_te, preds, zero_division=0)
# Asignación de variable o resultado intermedio
    auc  = roc_auc_score(y_te, probs) if probs is not None else np.nan

    print(f"== {name} ==")
# Asignación de variable o resultado intermedio
    print(f"Accuracy={acc:.4f}  Precision={prec:.4f}  Recall={rec:.4f}  F1={f1:.4f}  AUC={auc if not np.isnan(auc) else 'N/A'}\n")
# Asignación de variable o resultado intermedio
    print(classification_report(y_te, preds, digits=4))

# Asignación de variable o resultado intermedio
    cm = confusion_matrix(y_te, preds)
# Asignación de variable o resultado intermedio
    plt.figure(figsize=(4,3))
# Asignación de variable o resultado intermedio
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Matriz de confusión — {name}")
    plt.xlabel('Predicho')
    plt.ylabel('Real')
    plt.show()

# Condición para ejecutar código según un criterio
    if probs is not None:
# Asignación de variable o resultado intermedio
        fpr, tpr, _ = roc_curve(y_te, probs)
# Asignación de variable o resultado intermedio
        plt.figure(figsize=(5,4))
        plt.plot(fpr, tpr)
# Asignación de variable o resultado intermedio
        plt.plot([0,1],[0,1],'--', alpha=0.5)
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.title(f'ROC curve — {name}')
        plt.show()

# Bucle para iterar sobre datos
for name, model in models.items():
    print('Entrenando', name)
    model.fit(X_train, y_train)
    eval_and_plot(name, model, X_test, y_test)

# --- Fin de celda ---

In [ ]:
# --- Inicio de celda: explicación de la lógica de esta sección ---
# 7) Importancia de variables (RandomForest) y coeficientes (Logistic Regression)
# Obtener nombres de features tras ColumnTransformer de forma robusta
# Definición de una función
def get_feature_names_from_columntransformer(ct, num_cols, cat_cols):
    # num_cols are passed through first
# Asignación de variable o resultado intermedio
    feature_names = []
    # numeric names
    feature_names.extend(list(num_cols))
    # categorical names from OHE
    try:
# Asignación de variable o resultado intermedio
        ohe_step = ct.named_transformers_['cat']
        # get_feature_names_out may require passing input features depending on sklearn version
        try:
# Asignación de variable o resultado intermedio
            cat_names = list(ohe_step.get_feature_names_out(cat_cols))
        except TypeError:
# Asignación de variable o resultado intermedio
            cat_names = list(ohe_step.get_feature_names_out())
        feature_names.extend(cat_names)
    except Exception as e:
        # fallback: create simple placeholders
# Bucle para iterar sobre datos
        feature_names.extend([f'cat_{i}' for i in range(len(cat_cols))])
    return feature_names

# RandomForest
# Asignación de variable o resultado intermedio
rf = models['RandomForest']
# Asignación de variable o resultado intermedio
prep_rf = rf.named_steps['prep']
# Asignación de variable o resultado intermedio
rf_clf = rf.named_steps['clf']
# Asignación de variable o resultado intermedio
feature_names_rf = get_feature_names_from_columntransformer(prep_rf, num_cols, cat_cols)
# Importación de librerías necesarias
importances = rf_clf.feature_importances_
# Importación de librerías necesarias
imp_df = pd.DataFrame({'feature': feature_names_rf, 'importance': importances}).sort_values('importance', ascending=False)
print('Top 20 features (RandomForest):')
display(imp_df.head(20))

# Logistic Regression
# Asignación de variable o resultado intermedio
lr = models['LogisticRegression']
# Asignación de variable o resultado intermedio
prep_lr = lr.named_steps['prep']
# Asignación de variable o resultado intermedio
lr_clf = lr.named_steps['clf']
# Asignación de variable o resultado intermedio
feature_names_lr = get_feature_names_from_columntransformer(prep_lr, num_cols, cat_cols)
# Asignación de variable o resultado intermedio
coefs = lr_clf.coef_.ravel()
# Asignación de variable o resultado intermedio
coef_df = pd.DataFrame({'feature': feature_names_lr, 'coef': coefs}).assign(abs_coef=lambda d: d['coef'].abs()).sort_values('abs_coef', ascending=False)
print('Top 20 coefficients (LogisticRegression):')
display(coef_df.head(20))
# --- Fin de celda ---

## Conclusiones y siguientes pasos
- Revisa las variables con mayor importancia y prueba modelos focalizados o técnicas de reducción/selección.
- Considera balancear clases (SMOTE o undersampling) y re-evaluar recall/AUC si la clase positiva es minoritaria.
- Implementar despliegue y monitoreo: recalibrar periodicámente, usar alertas para clientes con alta probabilidad de churn.
